# PyOPV

## Installation

## Example usage

In [ ]:
# cov

### Importing

In [1]:
import sys; sys.path.append('..') # add parent directory to path (for testing)
import pyopv

## Getting latest DICOM Standards

In [2]:
from pyopv import get_dicom_standard # if you run this cell, you will get the latest SAP DICOM standard
get_dicom_standard() # if you run this cell, you will get the latest SAP DICOM standard and writes in a csv file in the same directory as this notebook

Up to date DICOM standard json files successfully extracted.
Up to date DICOM attributes successfully extracted for CIOD: ophthalmic-visual-field-static-perimetry-measurements.
Up to date DICOM attributes successfully written to /ophthalmic-visual-field-static-perimetry-measurements.csv.
Headers:
           tag                        name                   keyword  \
1  (0008,0005)      Specific Character Set      SpecificCharacterSet   
5  (0008,0012)      Instance Creation Date      InstanceCreationDate   
6  (0008,0013)      Instance Creation Time      InstanceCreationTime   
7  (0008,0014)        Instance Creator UID        InstanceCreatorUID   
8  (0008,0015)  Instance Coercion DateTime  InstanceCoercionDateTime   

  valueRepresentation valueMultiplicity retired        id  
1                  CS               1-n       N  00080005  
5                  DA                 1       N  00080012  
6                  TM                 1       N  00080013  
7                  UI        

### Checking missing tags

A single file

In [4]:
from pyopv import OPVDicom

file_path = '/Users/shallaj/Library/CloudStorage/OneDrive-UniversityofCalifornia,SanDiegoHealth/VF Data Standards/opv_dicom_omop/examples/sample_files/SD1031_20220908_OD_OPV.dcm'
m_opvdicom = pyopv.read_dicom(file_path)
missing_tags_df, incorrect_tags_df= m_opvdicom.check_dicom_compliance() # check if the dicom file is compliant with the standard
display(missing_tags_df)
display(incorrect_tags_df)

,name,description,type,vm,vr,tag
0,DataSetName,The name assigned to the data set.,1,1.0,LO,"(0024, 0306)"
1,DataSetVersion,The software version identifier assigned to th...,1,1.0,LO,"(0024, 0307)"
2,DataSetSource,Source of the data set e.g. the name of the ma...,1,1.0,LO,"(0024, 0308)"
3,DataSetDescription,Description of the data set.,3,1.0,LO,"(0024, 0309)"
4,AlgorithmFamilyCodeSequence,The family of algorithm(s) that best describes...,1,1.0,SQ,"(0066, 002F)"
...,...,...,...,...,...,...
73,PupilSize,The horizontal diameter measurement of the pup...,2,1.0,FD,"(0046, 0044)"
74,PupilDilated,The patient's pupils were pharmacologically di...,2,1.0,CS,"(0022, 000D)"
75,IntraOcularPressure,Value of intraocular pressure in mmHg.,3,1.0,FL,"(0022, 000B)"
76,VisualAcuityMeasurementSequence,Measurements of a patient's visual acuity.,3,1.0,SQ,"(0024, 0110)"


""


Entire directory

In [3]:
m_opvdicoms = pyopv.read_dicom_directory('/Users/shallaj/Library/CloudStorage/OneDrive-UniversityofCalifornia,SanDiegoHealth/VF Data Standards/opv_dicom_omop/examples/sample_files', file_extension='dcm')
missingtags_df = m_opvdicoms.check_dicom_compliance()
missingtags_df.head(5)



/Users/shallaj/Library/CloudStorage/OneDrive-UniversityofCalifornia,SanDiegoHealth/VF Data Standards/PyOPV Paper/py-opv/examples/../pyopv/opvdicom/__init__.py:188: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  missing_tags_df = pd.concat([missing_tags_df, pd.DataFrame({


,File Name,Missing tags Count / Missing DICOM Meta Information Header,Number of Missing Required Tags,Percentage of Missing Required Tags
0,ops_opv.dcm,93,75,73.53
1,test_716621_opv_od.dcm,82,63,61.76
2,SD1031_20230914_OS_OPV.dcm,81,63,61.76
3,SD1031_20230914_OD_OPV.dcm,81,63,61.76
4,SD1031_20220908_OD_OPV.dcm,78,62,60.78


## Converting OPV DICOM to Pandas DataFrame

*#* This snippet converts the entire dicom file into a very long pandas DataFrame

In [6]:

from pyopv import dicom_to_dataframe

file_path = '/Users/shallaj/Library/CloudStorage/OneDrive-UniversityofCalifornia,SanDiegoHealth/VF Data Standards/opv_dicom_omop/examples/sample_files/SD1031_20230914_OD_OPV.dcm'  # add file path here e.g. data/MY_OPV_FILE.dcm
dicom_df = dicom_to_dataframe(file_path)
dicom_df.head(3)


,0,1,2,3,4,5,6,7,8,9,...,849,850,851,852,853,854,855,856,857,858
name,Specific Character Set,Instance Creation Date,Instance Creation Time,SOP Class UID,SOP Instance UID,Study Date,Study Time,Accession Number,Modality,Manufacturer,...,Concept Name Code Sequence,Code Value,Coding Scheme Designator,Code Meaning,Private Creator,Private tag data,Private Creator,Private tag data,Private tag data,Private tag data
value,ISO_IR 192,20231115,115903.087,1.2.840.10008.5.1.4.1.1.80.1,1.2.276.0.75.2.5.80.25.3.231115115903087.34504...,20230914,103811.519,,OPV,Carl Zeiss Meditec,...,"[[(0008, 0100) Code Value ...",R-408C3,SRT,Diagnostic,99CZM_Hfa_OphthalmicVisualFieldStaticPerimetry,Central,99CZM_NIM_INTERNAL_01,HfaOphthalmicVisualFieldStaticPerimetryMeasure...,2.13,HfaVisualFieldStaticPerimetryTestResults\HfaVi...


# JSON conversion
* this snippet will be updated in future versions to accomodate FHIR IG because that IG is not currently available. 

In [8]:
import pydicom

file_path =  '/Users/shallaj/Library/CloudStorage/OneDrive-UniversityofCalifornia,SanDiegoHealth/VF Data Standards/opv_dicom_omop/examples/sample_files/SD1031_20230914_OD_OPV.dcm'  # add file path here e.g. data/MY_OPV_FILE.dcm
ds = pydicom.dcmread(file_path)
json = ds.to_json_dict()
json['00080080']['Value']

['UCSD Ophthalmology Research']

In [9]:
# get nested values
json = ds.to_json_dict()
points = json['00240089']['Value'][0]['00240097']
points

{'vr': 'SQ',
 'Value': [{'00240092': {'vr': 'FL', 'Value': [-23.0]},
   '00240100': {'vr': 'FL', 'Value': [1.0]},
   '00240102': {'vr': 'CS', 'Value': ['YES']},
   '00240103': {'vr': 'FL', 'Value': [-20.0]},
   '00240104': {'vr': 'FL', 'Value': [1.0]}}]}

### Getting pointwise data

Single file

In [10]:
m_opvdicom = pyopv.read_dicom('/Users/shallaj/Library/CloudStorage/OneDrive-UniversityofCalifornia,SanDiegoHealth/VF Data Standards/opv_dicom_omop/examples/sample_files/SD1031_20230914_OD_OPV.dcm')

pointwise_data = m_opvdicom.pointwise_to_pandas()
pointwise_data.head(5)

,person_id,sop_instance_uid,study_instance_uid,laterality,x_coords,y_coords,stimulus_result,sensitivity_values,age_corrected_sensitivity_deviation_values,age_corrected_sensitivity_deviation_probability_values,generalized_defect_corrected_sensitivity_deviation_flag,generalized_defect_corrected_sensitivity_values,generalized_defect_corrected_sensitivity_probability_values
0,SD1031,1.2.276.0.75.2.5.80.25.3.231115115903087.34504...,1.2.276.0.75.2.2.30.2.1.20230914103811.860.15084,R,-1.0,9.0,SEEN,7.0,-23.0,1.0,YES,-20.0,1.0
1,SD1031,1.2.276.0.75.2.5.80.25.3.231115115903087.34504...,1.2.276.0.75.2.2.30.2.1.20230914103811.860.15084,R,1.0,9.0,SEEN,20.0,-9.0,1.0,YES,-6.0,2.0
2,SD1031,1.2.276.0.75.2.5.80.25.3.231115115903087.34504...,1.2.276.0.75.2.2.30.2.1.20230914103811.860.15084,R,-5.0,7.0,SEEN,26.0,-4.0,5.0,YES,-1.0,0.0
3,SD1031,1.2.276.0.75.2.5.80.25.3.231115115903087.34504...,1.2.276.0.75.2.2.30.2.1.20230914103811.860.15084,R,-3.0,7.0,SEEN,25.0,-6.0,1.0,YES,-2.0,0.0
4,SD1031,1.2.276.0.75.2.5.80.25.3.231115115903087.34504...,1.2.276.0.75.2.2.30.2.1.20230914103811.860.15084,R,-1.0,7.0,SEEN,23.0,-7.0,1.0,YES,-4.0,2.0


Entire directory

In [12]:
m_opvdicoms = pyopv.read_dicom_directory('/Users/shallaj/Library/CloudStorage/OneDrive-UniversityofCalifornia,SanDiegoHealth/VF Data Standards/opv_dicom_omop/examples/sample_files', file_extension='dcm')

pointwise_data, error_df = m_opvdicoms.pointwise_to_pandas()
pointwise_data.head(3)

,person_id,sop_instance_uid,study_instance_uid,laterality,x_coords,y_coords,stimulus_result,sensitivity_values,age_corrected_sensitivity_deviation_values,age_corrected_sensitivity_deviation_probability_values,generalized_defect_corrected_sensitivity_deviation_flag,generalized_defect_corrected_sensitivity_values,generalized_defect_corrected_sensitivity_probability_values
0,503286,2.25.131336704941307872492978734319609842773,2.25.330722060978132347997218568577131398170,L,3.0,3.0,SEEN,0.0,-30.4,0.5,YES,-9.415,0.5
1,503286,2.25.131336704941307872492978734319609842773,2.25.330722060978132347997218568577131398170,L,9.0,3.0,SEEN,0.0,-30.511,0.5,YES,-9.526,0.5
2,503286,2.25.131336704941307872492978734319609842773,2.25.330722060978132347997218568577131398170,L,21.0,3.0,SEEN,0.0,-29.733,0.5,YES,-8.748,0.5


In [13]:
import pandas as pd

# Assuming 'pointwise_data' is already defined as a DataFrame
x_coord = pointwise_data['x_coords']
y_coord = pointwise_data['y_coords']

ns_pointwise_data = pointwise_data.copy()

# Mapping to NS system
# Flip x coordinates for left laterality
ns_pointwise_data.loc[ns_pointwise_data['laterality'] == 'L', 'x_coords'] = -1 * ns_pointwise_data['x_coords']

# Function to map x coordinates to NS system
def map_ns_x(row):
    x_coord = int(row['x_coords'])
    if x_coord > 0:
        return f'T{abs(x_coord)}_'
    else:
        return f'N{abs(x_coord)}_'

# Apply the function to create the 'ns' column
ns_pointwise_data['ns'] = ns_pointwise_data.apply(map_ns_x, axis=1)

# Function to map y coordinates to IS system
def map_ns_y(y):
    y_coord = int(y)
    if y_coord < 0:
        return f'I{abs(y_coord)}'
    else:
        return f'S{y_coord}'

# Apply the function to append y coordinates to the 'ns' column
ns_pointwise_data['ns'] = ns_pointwise_data['ns'] + ns_pointwise_data['y_coords'].apply(map_ns_y)

# Display the updated DataFrame
ns_pointwise_data.head(3)


,person_id,sop_instance_uid,study_instance_uid,laterality,x_coords,y_coords,stimulus_result,sensitivity_values,age_corrected_sensitivity_deviation_values,age_corrected_sensitivity_deviation_probability_values,generalized_defect_corrected_sensitivity_deviation_flag,generalized_defect_corrected_sensitivity_values,generalized_defect_corrected_sensitivity_probability_values,ns
0,503286,2.25.131336704941307872492978734319609842773,2.25.330722060978132347997218568577131398170,L,-3.0,3.0,SEEN,0.0,-30.4,0.5,YES,-9.415,0.5,N3_S3
1,503286,2.25.131336704941307872492978734319609842773,2.25.330722060978132347997218568577131398170,L,-9.0,3.0,SEEN,0.0,-30.511,0.5,YES,-9.526,0.5,N9_S3
2,503286,2.25.131336704941307872492978734319609842773,2.25.330722060978132347997218568577131398170,L,-21.0,3.0,SEEN,0.0,-29.733,0.5,YES,-8.748,0.5,N21_S3


In [14]:
# Dictionary mapping original column names to Humphrey Visual Field equivalents
opv_hfa_dict = {
    'sensitivity_values': 'threshold',
    'age_corrected_sensitivity_deviation_values': 'total_deviation',
    'age_corrected_sensitivity_deviation_probability_values': 'total_deviation_probability',
    'generalized_defect_corrected_sensitivity_deviation_flag': 'generalized_defect_threshold_deviation_flag',
    'generalized_defect_corrected_sensitivity_values': 'pattern_deviation',
    'generalized_defect_corrected_sensitivity_probability_values': 'pattern_deviation_probability',
    'ns': 'nasal_temporal_superior_inferior'
}

# Assuming 'pointwise_data' is your DataFrame, you can rename the columns using this dictionary
ns_pointwise_data.rename(columns=opv_hfa_dict, inplace=True)

# Display the modified DataFrame to ensure the changes are correct
ns_pointwise_data.head(3)


,person_id,sop_instance_uid,study_instance_uid,laterality,x_coords,y_coords,stimulus_result,threshold,total_deviation,total_deviation_probability,generalized_defect_threshold_deviation_flag,pattern_deviation,pattern_deviation_probability,nasal_temporal_superior_inferior
0,503286,2.25.131336704941307872492978734319609842773,2.25.330722060978132347997218568577131398170,L,-3.0,3.0,SEEN,0.0,-30.4,0.5,YES,-9.415,0.5,N3_S3
1,503286,2.25.131336704941307872492978734319609842773,2.25.330722060978132347997218568577131398170,L,-9.0,3.0,SEEN,0.0,-30.511,0.5,YES,-9.526,0.5,N9_S3
2,503286,2.25.131336704941307872492978734319609842773,2.25.330722060978132347997218568577131398170,L,-21.0,3.0,SEEN,0.0,-29.733,0.5,YES,-8.748,0.5,N21_S3


In [15]:
# mapping humphrey equivalent to ucsd equivalent
hfa_ucsd_dict = {
    'threshold': 'Thr',
    'total_deviation': 'TD',
    'total_deviation_probability': 'TDP',
    'generalized_defect_threshold_deviation_flag': 'generalized_defect_corrected_sensitivity_deviation_flag',
    'pattern_deviation': 'PD',
    'pattern_deviation_probability': 'PDP',
    'nasal_temporal_superior_inferior': 'ns'
}

# Assuming 'pointwise_data' is your DataFrame, you can rename the columns using this dictionary
ns_pointwise_data.rename(columns=hfa_ucsd_dict, inplace=True)

# Display the modified DataFrame to ensure the changes are correct
ns_pointwise_data.head(3)

,person_id,sop_instance_uid,study_instance_uid,laterality,x_coords,y_coords,stimulus_result,Thr,TD,TDP,generalized_defect_corrected_sensitivity_deviation_flag,PD,PDP,ns
0,503286,2.25.131336704941307872492978734319609842773,2.25.330722060978132347997218568577131398170,L,-3.0,3.0,SEEN,0.0,-30.4,0.5,YES,-9.415,0.5,N3_S3
1,503286,2.25.131336704941307872492978734319609842773,2.25.330722060978132347997218568577131398170,L,-9.0,3.0,SEEN,0.0,-30.511,0.5,YES,-9.526,0.5,N9_S3
2,503286,2.25.131336704941307872492978734319609842773,2.25.330722060978132347997218568577131398170,L,-21.0,3.0,SEEN,0.0,-29.733,0.5,YES,-8.748,0.5,N21_S3


Writing pointwise data into JSON
* Single file

In [17]:
from pyopv import pointwise_to_nested_json
file_path = '/Users/shallaj/Library/CloudStorage/OneDrive-UniversityofCalifornia,SanDiegoHealth/VF Data Standards/opv_dicom_omop/examples/sample_files/SD1031_20230914_OD_OPV.dcm' # add file path here e.g. data/MY_OPV_FILE.dcm
nested_json = pointwise_to_nested_json(file_path)
nested_json['SD1031']['R']['1.2.276.0.75.2.5.80.25.3.231115115903087.345048637379.415558310'][0]

{'study_instance_uid': '1.2.276.0.75.2.2.30.2.1.20230914103811.860.15084',
 'x_coords': -1.0,
 'y_coords': 9.0,
 'stimulus_result': 'SEEN',
 'sensitivity_values': 7.0,
 'age_corrected_sensitivity_deviation_values': -23.0,
 'age_corrected_sensitivity_deviation_probability_values': 1.0,
 'generalized_defect_corrected_sensitivity_deviation_flag': 'YES',
 'generalized_defect_corrected_sensitivity_values': -20.0,
 'generalized_defect_corrected_sensitivity_probability_values': 1.0}

# Example conversion of a JSON resource to pandas DataFrame

In [18]:
import pandas as pd
pd.json_normalize(nested_json['SD1031']['R']['1.2.276.0.75.2.5.80.25.3.231115115903087.345048637379.415558310'], sep='_').head(3)

,study_instance_uid,x_coords,y_coords,stimulus_result,sensitivity_values,age_corrected_sensitivity_deviation_values,age_corrected_sensitivity_deviation_probability_values,generalized_defect_corrected_sensitivity_deviation_flag,generalized_defect_corrected_sensitivity_values,generalized_defect_corrected_sensitivity_probability_values
0,1.2.276.0.75.2.2.30.2.1.20230914103811.860.15084,-1.0,9.0,SEEN,7.0,-23.0,1.0,YES,-20.0,1.0
1,1.2.276.0.75.2.2.30.2.1.20230914103811.860.15084,1.0,9.0,SEEN,20.0,-9.0,1.0,YES,-6.0,2.0
2,1.2.276.0.75.2.2.30.2.1.20230914103811.860.15084,-5.0,7.0,SEEN,26.0,-4.0,5.0,YES,-1.0,0.0


* Bulk data

In [21]:

from pyopv import opvdicoms_pointwise_to_nested_json 

directory_path = '/Users/shallaj/Library/CloudStorage/OneDrive-UniversityofCalifornia,SanDiegoHealth/VF Data Standards/opv_dicom_omop/examples/sample_files' # add directory path here e.g. data

# Run the function and get the nested JSON
bulk_pointwise_nested_json = opvdicoms_pointwise_to_nested_json(directory_path)

# convert the nested JSON so that the keys are the patient IDs


bulk_pointwise_nested_json['SD1031']['R']['1.2.276.0.75.2.5.80.25.3.221018115130492.345048637379.430073977'][0]

Error processing file /Users/shallaj/Library/CloudStorage/OneDrive-UniversityofCalifornia,SanDiegoHealth/VF Data Standards/opv_dicom_omop/examples/sample_files/ops_opv.dcm: (0024, 0097)


{'study_instance_uid': '1.2.276.0.75.2.2.30.2.1.20220908090351.860.15084',
 'x_coords': -9.0,
 'y_coords': 21.0,
 'stimulus_result': 'SEEN',
 'sensitivity_values': 27.0,
 'age_corrected_sensitivity_deviation_values': 1.0,
 'age_corrected_sensitivity_deviation_probability_values': 0.0,
 'generalized_defect_corrected_sensitivity_deviation_flag': 'YES',
 'generalized_defect_corrected_sensitivity_values': 0.0,
 'generalized_defect_corrected_sensitivity_probability_values': 0.0}

In [22]:
import pandas as pd
# pd.normalize the nested JSON
bulk_pointwise_nested_json['503286']['L']['2.25.131336704941307872492978734319609842773'][0]

{'study_instance_uid': '2.25.330722060978132347997218568577131398170',
 'x_coords': 3.0,
 'y_coords': 3.0,
 'stimulus_result': 'SEEN',
 'sensitivity_values': 0.0,
 'age_corrected_sensitivity_deviation_values': -30.399999618530273,
 'age_corrected_sensitivity_deviation_probability_values': 0.5,
 'generalized_defect_corrected_sensitivity_deviation_flag': 'YES',
 'generalized_defect_corrected_sensitivity_values': -9.414999961853027,
 'generalized_defect_corrected_sensitivity_probability_values': 0.5}

In [23]:
pd.json_normalize(bulk_pointwise_nested_json['503286']['L']['2.25.131336704941307872492978734319609842773']).head(3)

,study_instance_uid,x_coords,y_coords,stimulus_result,sensitivity_values,age_corrected_sensitivity_deviation_values,age_corrected_sensitivity_deviation_probability_values,generalized_defect_corrected_sensitivity_deviation_flag,generalized_defect_corrected_sensitivity_values,generalized_defect_corrected_sensitivity_probability_values
0,2.25.330722060978132347997218568577131398170,3.0,3.0,SEEN,0.0,-30.4,0.5,YES,-9.415,0.5
1,2.25.330722060978132347997218568577131398170,9.0,3.0,SEEN,0.0,-30.511,0.5,YES,-9.526,0.5
2,2.25.330722060978132347997218568577131398170,21.0,3.0,SEEN,0.0,-29.733,0.5,YES,-8.748,0.5
